# DeepAR - Experiments

### DeepAR (Salinas et al., 2017)

The **DeepAR** algorithm uses autoregressive features that are used as the inputs to a recurrent neural network. The RNN uses LSTM cells as default but can also changed to GRU cells. In both cases previous time points are taken as input. 

- [`DeepAREestimator`](https://gluon-ts.mxnet.io/api/gluonts/gluonts.model.deepar.html)

In [1]:
!pip install gluonts

    100% |████████████████████████████████| 296kB 14.8MB/s ta 0:00:01
    100% |████████████████████████████████| 4.8MB 10.5MB/s ta 0:00:011
    100% |████████████████████████████████| 28.4MB 1.9MB/s  eta 0:00:01    93% |██████████████████████████████  | 26.7MB 102.9MB/s eta 0:00:01
    100% |████████████████████████████████| 81kB 42.2MB/s ta 0:00:01
    100% |████████████████████████████████| 10.4MB 6.1MB/s eta 0:00:01
    100% |████████████████████████████████| 61kB 43.3MB/s ta 0:00:01
    100% |████████████████████████████████| 194kB 63.7MB/s eta 0:00:01
  Running setup.py bdist_wheel for holidays ... done
  Stored in directory: /home/ec2-user/.cache/pip/wheels/40/a9/2b/94ac5464363d37564a87dc93a9d21a5850aac14a4608197003
  Running setup.py bdist_wheel for ujson ... done
  Stored in directory: /home/ec2-user/.cache/pip/wheels/28/77/e4/0311145b9c2e2f01470e744855131f9e34d6919687550f87d1
Successfully built holidays ujson
  Found existing installation: pandas 0.24.2
    Uninstalling panda

In [2]:
# standard imports
import numpy as np
import pandas as pd

# json
import json

# gluon data
from gluonts.dataset.repository.datasets import get_dataset, dataset_recipes
from gluonts.dataset.util import to_pandas

# gluon imports
from gluonts.trainer import Trainer
from gluonts.evaluation.backtest import make_evaluation_predictions
from gluonts.evaluation import Evaluator

# model imports
# from gluonts.model.deepstate import DeepStateEstimator
# from gluonts.model.deep_factor import DeepFactorEstimator
from gluonts.model.deepar import DeepAREstimator

import mxnet as mx
from pprint import pprint


INFO:root:Using CPU


In [3]:
def deepar(data="m4_monthly", seed=42, epochs=100, batches=50):
    
    dataset = get_dataset(data, regenerate=False)
    mx.random.seed(seed)
    np.random.seed(seed)
    
    trainer = Trainer(
        ctx=mx.cpu(0),
        epochs=epochs,      
        num_batches_per_epoch=batches,      
        learning_rate=1e-3,
    )
    
    cardinality = int(dataset.metadata.feat_static_cat[0].cardinality)
    estimator = DeepAREstimator(
        trainer=trainer,
        cardinality=[cardinality],
#         context_length=dataset.metadata.prediction_length,
        prediction_length=dataset.metadata.prediction_length,
        freq=dataset.metadata.freq,
        use_feat_static_cat=True   
    )
        
    predictor = estimator.train(dataset.train)
    
    forecast_it, ts_it = make_evaluation_predictions(
        dataset.test, predictor=predictor, num_eval_samples=100
    )
    
    agg_metrics, item_metrics = Evaluator()(
        ts_it, forecast_it, num_series=len(dataset.test)
    )
    
    metrics = ["MASE", "sMAPE", "MSIS", "wQuantileLoss[0.5]", "wQuantileLoss[0.9]"]
    output = {key: round(value, 8) for key, value in agg_metrics.items() if key in metrics}
    output["epochs"] = epochs
    output["seed"] = seed
    return(output)


In [4]:
%%time
res = deepar(data="m4_monthly", seed=43, epochs=200, batches=100)
pprint(res)

INFO:root:downloading and processing m4_monthly


saving time-series into /home/ec2-user/.mxnet/gluon-ts/datasets/m4_monthly/train/data.json
saving time-series into /home/ec2-user/.mxnet/gluon-ts/datasets/m4_monthly/test/data.json


INFO:root:Start model training
INFO:root:Number of parameters in DeepARTrainingNetwork: 973443
INFO:root:Epoch[0] Learning rate is 0.001
100%|██████████| 100/100 [00:04<00:00, 22.58it/s, avg_epoch_loss=7.51]
INFO:root:Epoch[0] Elapsed time 4.432 seconds
INFO:root:Epoch[0] Evaluation metric 'epoch_loss'=7.514194
INFO:root:Epoch[1] Learning rate is 0.001
100%|██████████| 100/100 [00:04<00:00, 24.86it/s, avg_epoch_loss=7.67]
INFO:root:Epoch[1] Elapsed time 4.023 seconds
INFO:root:Epoch[1] Evaluation metric 'epoch_loss'=7.674707
INFO:root:Epoch[2] Learning rate is 0.001
100%|██████████| 100/100 [00:04<00:00, 24.00it/s, avg_epoch_loss=6.92]
INFO:root:Epoch[2] Elapsed time 4.169 seconds
INFO:root:Epoch[2] Evaluation metric 'epoch_loss'=6.915264
INFO:root:Epoch[3] Learning rate is 0.001
100%|██████████| 100/100 [00:03<00:00, 25.19it/s, avg_epoch_loss=6.78]
INFO:root:Epoch[3] Elapsed time 3.973 seconds
INFO:root:Epoch[3] Evaluation metric 'epoch_loss'=6.776501
INFO:root:Epoch[4] Learning rate 

{'MASE': 1.03131091,
 'MSIS': 17.00587368,
 'epochs': 200,
 'sMAPE': 0.13666711,
 'seed': 43,
 'wQuantileLoss[0.5]': 0.12285705,
 'wQuantileLoss[0.9]': 0.08479357}
CPU times: user 1h 4min 32s, sys: 41.3 s, total: 1h 5min 14s
Wall time: 35min 28s


In [6]:
%%time
res = deepar(data="m4_monthly", seed=44, epochs=100, batches=500)
pprint(res)

INFO:root:using dataset already processed in path /home/ec2-user/.mxnet/gluon-ts/datasets/m4_monthly.
INFO:root:Start model training
INFO:root:Number of parameters in DeepARTrainingNetwork: 973443
INFO:root:Epoch[0] Learning rate is 0.001
100%|██████████| 500/500 [00:20<00:00, 23.92it/s, avg_epoch_loss=7.32]
INFO:root:Epoch[0] Elapsed time 20.903 seconds
INFO:root:Epoch[0] Evaluation metric 'epoch_loss'=7.324955
INFO:root:Epoch[1] Learning rate is 0.001
100%|██████████| 500/500 [00:19<00:00, 25.13it/s, avg_epoch_loss=6.82]
INFO:root:Epoch[1] Elapsed time 19.897 seconds
INFO:root:Epoch[1] Evaluation metric 'epoch_loss'=6.822595
INFO:root:Epoch[2] Learning rate is 0.001
100%|██████████| 500/500 [00:20<00:00, 24.84it/s, avg_epoch_loss=6.8] 
INFO:root:Epoch[2] Elapsed time 20.134 seconds
INFO:root:Epoch[2] Evaluation metric 'epoch_loss'=6.802164
INFO:root:Epoch[3] Learning rate is 0.001
100%|██████████| 500/500 [00:20<00:00, 24.65it/s, avg_epoch_loss=6.84]
INFO:root:Epoch[3] Elapsed time 2

{'MASE': 0.97483446,
 'MSIS': 16.89402796,
 'epochs': 100,
 'sMAPE': 0.13445326,
 'seed': 44,
 'wQuantileLoss[0.5]': 0.11874202,
 'wQuantileLoss[0.9]': 0.0730578}
CPU times: user 1h 45min 37s, sys: 1min 14s, total: 1h 46min 51s
Wall time: 54min 43s


In [12]:
# %%time
# results = []

# if __name__ == "__main__":
#     for i in range(42, 45):
#         print("Seed:", i)
#         res = deepar(data="m4_monthly", seed=i, epochs=100, batches=1000)
#         pprint(res)
#         results.append(res)
    
# results

### DeepAR - use_feat_static==False

In [7]:
def deepar_no_feat(data="m4_monthly", seed=42, epochs=100, batches=50):
    
    dataset = get_dataset(data, regenerate=False)
    mx.random.seed(seed)
    np.random.seed(seed)
    
    trainer = Trainer(
        ctx=mx.cpu(0),
        epochs=epochs,      
        num_batches_per_epoch=batches,      
        learning_rate=1e-3,
    )
    
#     cardinality = int(dataset.metadata.feat_static_cat[0].cardinality)
    estimator = DeepAREstimator(
        trainer=trainer,
#         cardinality=[cardinality],
        prediction_length=dataset.metadata.prediction_length,
        freq=dataset.metadata.freq,
#         use_feat_static_cat=True   
    )
        
    predictor = estimator.train(dataset.train)
    
    forecast_it, ts_it = make_evaluation_predictions(
        dataset.test, predictor=predictor, num_eval_samples=100
    )
    
    agg_metrics, item_metrics = Evaluator()(
        ts_it, forecast_it, num_series=len(dataset.test)
    )
    
    metrics = ["MASE", "sMAPE", "MSIS", "wQuantileLoss[0.5]", "wQuantileLoss[0.9]"]
    output = {key: round(value, 8) for key, value in agg_metrics.items() if key in metrics}
    output["epochs"] = epochs
    output["seed"] = seed
    return(output)


In [8]:
%%time
res = deepar_no_feat(data="m4_monthly", seed=42, epochs=100, batches=1000)
pprint(res)

INFO:root:using dataset already processed in path C:\Users\TM\.mxnet\gluon-ts\datasets\m4_monthly.
INFO:root:Start model training
INFO:root:Number of parameters in DeepARTrainingNetwork: 13463
INFO:root:Epoch[0] Learning rate is 0.001
100%|██████████| 1000/1000 [01:11<00:00, 13.97it/s, avg_epoch_loss=7.03]
INFO:root:Epoch[0] Elapsed time 71.576 seconds
INFO:root:Epoch[0] Evaluation metric 'epoch_loss'=7.031377
INFO:root:Epoch[1] Learning rate is 0.001
100%|██████████| 1000/1000 [01:09<00:00, 14.45it/s, avg_epoch_loss=6.9]
INFO:root:Epoch[1] Elapsed time 69.187 seconds
INFO:root:Epoch[1] Evaluation metric 'epoch_loss'=6.904093
INFO:root:Epoch[2] Learning rate is 0.001
100%|██████████| 1000/1000 [01:09<00:00, 14.46it/s, avg_epoch_loss=6.62]
INFO:root:Epoch[2] Elapsed time 69.186 seconds
INFO:root:Epoch[2] Evaluation metric 'epoch_loss'=6.615703
INFO:root:Epoch[3] Learning rate is 0.001
100%|██████████| 1000/1000 [01:09<00:00, 14.47it/s, avg_epoch_loss=6.62]
INFO:root:Epoch[3] Elapsed tim

{'MASE': 0.9894242,
 'MSIS': 9.79866835,
 'epochs': 100,
 'sMAPE': 0.13057258,
 'seed': 42,
 'wQuantileLoss[0.5]': 0.11604762,
 'wQuantileLoss[0.9]': 0.06786353}
Wall time: 11h 55min 14s
